# Mini Project: Working with LLM APIs

This project demonstrates how to interact with Large Language Model (LLM) APIs.
We send different prompts to multiple roles and models, then compare their outputs.


# Part 1: Sending Role-Based Prompts to Models


In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

## Function: Send Prompt with a Specific System Role

This function wraps the API call logic and allows you to set an explicit *system role* for the LLM before sending the main user prompt.

In [2]:
def SendPrompt(system_role, prompt, model_name):
    completion = client.chat.completions.create(
        extra_headers={},
        extra_body={},
        model= model_name,
        messages=[
                {
                "role": "system",
                f"content": "You are {system_role} AI assistant.",
                },
                {"role": "user", "content": prompt},
            ],
        )
    response = completion.choices[0].message.content
    return response

First, we **create a client** to connect to the OpenRouter server using the API key obtained from their website.

In [3]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

We load an **AI-generated email** (from Gemini 2.5 Pro) stored in a local `.txt` file.

In [ ]:
f = open("email.txt", 'r')
email_text = f.read()
f.close()

We define **four roles**, each with a predefined **prompt** relevant to its character.

In [31]:
role_action_map = {"a detective" : "How many words starts with an e in this email",
                   "a Sarcastic" : "Can you guess this random email is generated by which AI Model?" ,
                   "a Literary scholar" : "Rate this email and review it" ,
                   "a doctor" : "what disease the writer might have?"
                   }

We then **send each prompt based on its role** to the chosen model.

In [23]:
print(SendPrompt("a detective", f'{role_action_map["a detective"]} : {email_text}'))

There are **10 words** that start with the letter “e” in the email.


In [24]:
print(SendPrompt("a Sarcastic", f'{role_action_map["a Sarcastic"]} : {email_text}'))

Honestly, I’d wager the writer behind that mail was **ChatGPT‑based (the GPT‑4 family)**.

Here’s why:

| Clue | What it points to |
|------|------------------|
| **Highly polished, corporate‑style diction** (e.g., “Standardized Non‑Compliance Protocol,” “Q3 Summer period”) | GPT‑4 is known for its “business‑like” register, pulling in industry jargon naturally. |
| **Numerous, oddly specific references** (e.g., “Section 7, subsection 4b,” “Oct 1”). | GPT‑4 often introduces *invented* details that sound plausible but never exist, a signature of its “hallucination” style. |
| **Very structured, bulleted‑friendly layout** (subject line, To/From/Date header, sign‑off) | GPT‑4 produces clean, professional email formats routinely. |
| **Subtle “passive‑aggressive” tone** (“We trust this is simply an oversight”), balanced with precision (“Synergistically yours”) | GPT‑4 can mix nuance like that—common in its user‑generated emails. |

Everything else—especially the sense‑making of HOA bylaws, 

In [25]:
print(SendPrompt("a Literary scholar", f'{role_action_map["a Literary scholar"]} : {email_text}'))

**Overall Rating:** **5 / 10**

The message does hit all the technical points it tries to cover (policy reference, deadlines, consequences, and signatures), but the tone, word‑choice, and overall presentation make it feel more like a corporate memo than a friendly homeowner’s association notice. Below is a detailed review and some specific suggestions for tightening the communication.

---

## 1. Subject Line  
**Score: 4/10**  
*“Urgent Action Required: Review of Unsanctioned Decorative Gourd Placements (Policy 7.4b)”*

- The word “Urgent” feels clutch‑too‑strong for a seasonal décor matter.  
- “Review of” is misleading – the issue is already identified, not requiring review.  
- The parenthetical policy number adds corporate feel and is likely misunderstood by the average resident.

**Improvement:**  
`Please remove non‑seasonal gourds by **Friday, 5 p.m.**`  
or

`Seasonal décor reminder – gourds due away by Friday`

A clear, direct subject helps recipients grasp intent instantly.


In [26]:
print(SendPrompt("a doctor", f'{role_action_map["a doctor"]} : {email_text}'))

I’m not a medical professional, so I can’t diagnose a real disease here.  What I can do is note a few personality or behavioral patterns the tone of the note – patterns that people sometimes humorously (or sometimes seriously) link to certain “disorders” or syndromes.

| Pattern | Possible Personality or Behavioral Trait | “Disease” (in a humorous, non‑diagnostic sense) |
|---------|------------------------------------------|------------------------------------------------|
| Over‑formal, corporate‑style language used for a trivial neighborhood issue | High need for structure, control, and perfection in everyday life | “Corporate Narcissism” or “Executive Symptomatology” |
| Repetitive, slightly condescending “reminders” and bullet‑pointed rules | Tendency toward micromanagement or hyper‑ | “Hyper‑Rule‑Obsessive Condition” (HRO) |
| The writer’s insistence that everyone “return the gourds to a state of non‑visibility” | An intense need for conformity and fear of any deviation | “Confor

I send different prompts considering different role for AI bot for each time and then i write the response for each one to separate files

In [35]:
for role in role_action_map:
    response = SendPrompt(role, f"{role_action_map[role]} : {email_text}", "openai/gpt-oss-20b:free")
    with open(f"{role}.txt", "w", encoding="utf-8") as f:
        f.write(response)

The goal is to send the 4 responses to a different model with another role to summarize these 4 responses in a single one.

In [38]:
summarize_prompt = f"""summarize these four responses of models to an email in 10 lines:\n
                    """
i = 1
for role in role_action_map:
    f = open(f"{role}.txt", "r", encoding="utf-8")
    response_text = f.read()
    i += 1
    f.close()
    summarize_prompt += f"response {i} : {response_text}\n"

summarize_response = SendPrompt("a Summarizer", summarize_prompt, "google/gemini-2.0-flash-exp:free")
with open("summarized_response.txt", "w", encoding="utf-8") as f:
        f.write(summarize_response)

print(summarize_response)

Here's a 10-line summary of the provided model responses:

The first model determined that the "gourd email" contains 9 words starting with "e".
The second model suggests the email was likely generated by OpenAI's ChatGPT-3.5 due to its overly formal tone, repetitive phrasing,verbose sentences, lack of humor, and quirks in calendar terminology. Newer models like GPT-4, Claude, and Gemini would likely produce a more refined or coherent message.
The third model gives the email a 6.5/10, highlighting its clear subject line and instructions but criticizing its passive-aggressive diction, verbose style, and unnecessary drama. It suggests a more friendly and direct approach in a revised version.
The fourth and final model suggest the writer displays many clear traits of obsessive-compulsive personality disorder (OCPD) with some minor features of narcissistic personality disorder. 
The model makes clear that the email traits are behavioural patterns rather than a medical illness and professio

# Part 2: Implementing Memory in the Chatbot

Now, we want to **add memory capabilities** to the chatbot. There are several strategies for this, such as using a fixed-size array, a sliding window, or summarizing the conversation. In this implementation, we use the *summarized conversation* method to preserve context.

In [ ]:
Client_with_conversational_mem = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)
def get_summary(history, model_name):
    prompt = f"""Summarize the following conversation concisely. Capture the key facts, names, and topics discussed:
    {history}
    Summary:
    """
    completion = Client_with_conversational_mem.chat.completions.create(
        extra_headers={},
        extra_body={},
        model= model_name,
        messages=[
                {"role": "user", "content": prompt},
            ],
        )
    response = completion.choices[0].message.content
    return response



In [7]:

def sendPromptWithMem(conversation_history, model_name):
    completion = Client_with_conversational_mem.chat.completions.create(
        model = model_name,
        messages = conversation_history
        )
    response = completion.choices[0].message.content
    return response

In [11]:
system_prompt = {"role": "system", "content": "You are an AI assistant."}
conversation_history = []
conversation_history.append(system_prompt)
mem_summary = ""
summary_threshold = 3
user_prompt = ""
i = 1
while user_prompt != "EXIT":
    user_prompt = input("Send: ")
    print("**USER PROMPT**\n" + user_prompt + "\n\n")
    conversation_history.append({"role" : "user", "content": user_prompt})
    response = sendPromptWithMem(conversation_history, "openai/gpt-oss-20b:free")
    print("***AI Response\n" + response + "\n\n")
    conversation_history.append({"role" : "assistant", "content": response})
    i += 1
    if i > summary_threshold:
        mem_summary = get_summary(conversation_history, "openai/gpt-oss-20b:free")
        conversation_history = []
        conversation_history.append({"role" : "assistant", "content": mem_summary})
        i = 1

**USER PROMPT**
hello i'm Borna


***AI Response
Hello Borna! 👋 How can I help you today? Whether you have a question, need a brainstorm, or just want to chat, I'm all ears.


**USER PROMPT**
Guess my nationality


***AI Response
Sure! Based on the name “Borna,” my best guess is that you’re from a Slavic‑speaking country—most commonly Croatia or Slovenia (it’s also used in Bosnia‑Herzegovina and Serbia). Do any of those sound right? If not, feel free to let me know where you’re from!


**USER PROMPT**
no i'ts not right


***AI Response
Sure thing! I’d love to get a better hit this time. Could you give me a quick hint? Maybe something about your hometown, a hobby you’re really into, or a favorite food that’s tied to your country? That’ll help me narrow it down.


**USER PROMPT**
i live in middle east


***AI Response
Nice to hear you’re in the Middle East!  
The region covers a lot of countries—Saudi Arabia, UAE, Qatar, Bahrain, Kuwait, Oman, Jordan, Lebanon, Syria, Iraq, Yemen, Turkey,